In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [18]:
train_df = pd.read_csv('./train.csv')

In [19]:
train_df.head(10)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
5,20090312431273200,5,3423,person_click,basic,0,NaN,-412.991405,-157.314686,381.0,492.0,NaN,"Sure thing, Jo. Grab your notebook and come up...",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
6,20090312431273200,6,5197,person_click,basic,0,NaN,478.485079,-199.971679,593.0,485.0,NaN,"See you later, Teddy.",teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
7,20090312431273200,7,6180,person_click,basic,0,NaN,503.355128,-168.619913,609.0,453.0,NaN,I get to go to Gramps's meeting!,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
8,20090312431273200,8,7014,person_click,basic,0,NaN,510.733442,-157.720642,615.0,442.0,NaN,Now where did I put my notebook?,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
9,20090312431273200,9,7946,person_click,basic,0,NaN,512.048005,-153.743631,616.0,438.0,NaN,\u00f0\u0178\u02dc\u00b4,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4


In [20]:
train_labels_df = pd.read_csv('./train_labels.csv')
test_df = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [89]:
train_labels_df

,session_id,correct,session,question
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1
...,...,...,...,...
424111,22100215342220508_q18,1,22100215342220508,18
424112,22100215460321130_q18,1,22100215460321130,18
424113,22100217104993650_q18,1,22100217104993650,18
424114,22100219442786200_q18,1,22100219442786200,18


In [109]:
train_labels_df["session"] = train_labels_df.session_id.apply(lambda x: int(x.split('_')[0]))
train_labels_df["question"] = train_labels_df.session_id.apply(lambda x: int(x.split('_')[-1][1:]))
train_labels_df

,session_id,correct,session,question
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1
...,...,...,...,...
424111,22100215342220508_q18,1,22100215342220508,18
424112,22100215460321130_q18,1,22100215460321130,18
424113,22100217104993650_q18,1,22100217104993650,18
424114,22100219442786200_q18,1,22100219442786200,18


In [23]:
train_df.info()
print()
print("*"*40)
print()
train_labels_df.info()
print()
print("*"*40)
print()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26296946 entries, 0 to 26296945
Data columns (total 20 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      int64  
 1   index           int64  
 2   elapsed_time    int64  
 3   event_name      object 
 4   name            object 
 5   level           int64  
 6   page            float64
 7   room_coor_x     float64
 8   room_coor_y     float64
 9   screen_coor_x   float64
 10  screen_coor_y   float64
 11  hover_duration  float64
 12  text            object 
 13  fqid            object 
 14  room_fqid       object 
 15  text_fqid       object 
 16  fullscreen      int64  
 17  hq              int64  
 18  music           int64  
 19  level_group     object 
dtypes: float64(6), int64(7), object(7)
memory usage: 3.9+ GB

****************************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424116 entries, 0 to 424115
Data columns (total 4 columns):
 #   Column      Non-Null Count 

In [24]:
from collections import Counter
text_df = train_df['text']
text_list = []
for i in text_df:
    text_list.append(i)
res = Counter(text_list)
res

Counter({'undefined': 355969,
         'Whatcha doing over there, Jo?': 24228,
         'Just talking to Teddy.': 24301,
         'I gotta run to my meeting!': 24338,
         'Can I come, Gramps?': 11791,
         'Sure thing, Jo. Grab your notebook and come upstairs!': 11787,
         'See you later, Teddy.': 11658,
         "I get to go to Gramps's meeting!": 11687,
         'Now where did I put my notebook?': 24208,
         '\\u00f0\\u0178\\u02dc\\u00b4': 35654,
         nan: 16679807,
         'I love these photos of me and Teddy!': 12501,
         'Found it!': 23921,
         'Gramps is in trouble for losing papers?': 17219,
         "This can't be right!": 17220,
         'Gramps is a great historian!': 8521,
         "Hmm. Button's still not working.": 9984,
         "Let's get started. The Wisconsin Wonders exhibit opens tomorrow!": 23802,
         'Who wants to investigate the shirt artifact?': 23793,
         "Not Leopold here. He's been losing papers lately.": 11497,
     

In [110]:
train_df = train_df.drop(columns=['text'])
# for i in range(len(train_df)):
#     if train_labels_df['session'] == train_df['session_id'][i]:

KeyError: "['text'] not found in axis"

In [108]:
labels_dict = train_labels_df.set_index(train_labels_df.columns[2]).to_dict()[train_labels_df.columns[1]]
labels_dict

{20090312431273200: 1,
 20090312433251036: 1,
 20090312455206810: 1,
 20090313091715820: 1,
 20090313571836404: 1,
 20090314035813970: 1,
 20090314121766812: 1,
 20090314221187252: 1,
 20090314363702160: 1,
 20090314441803444: 1,
 20090315081004164: 1,
 20090315085850788: 1,
 20090315101457836: 1,
 20090315170769824: 1,
 20090315270479452: 1,
 20090316152177500: 1,
 20090316190523732: 1,
 20090317080721164: 1,
 20090317111400710: 1,
 20090317414400068: 1,
 20090318021527530: 1,
 20090318322359292: 1,
 20090318541618010: 1,
 20090319012741024: 1,
 20090320140443148: 1,
 20100000291486630: 1,
 20100000404860470: 1,
 20100006432365430: 1,
 20100007011166932: 1,
 20100007445515820: 1,
 20100008103581056: 1,
 20100008262217530: 1,
 20100008572266050: 1,
 20100009003806170: 1,
 20100009303098716: 1,
 20100009314461780: 1,
 20100009385330468: 1,
 20100009560590796: 1,
 20100010540710908: 1,
 20100010554738616: 1,
 20100011003256550: 1,
 20100011033647636: 1,
 20100011050028640: 1,
 2010001127

In [113]:
for i in range(len(train_df))
if train_df['session_id'[i]] == labels_dict.keys():
    train_df['']

SyntaxError: expected ':' (3983102658.py, line 1)

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [27]:
numerics_df = train_df.drop(columns=['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid'])
category_df = train_df.drop(columns=['elapsed_time', 'level','page','room_coor_x','room_coor_y','screen_coor_x','screen_coor_y','hover_duration','fullscreen','hq','music'])

In [28]:
numerics_df

,session_id,index,elapsed_time,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fullscreen,hq,music,level_group,correct,question
0,20090312431273200,0,0,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,0,0,1,0-4,1.0,1.0
1,20090312431273200,1,1323,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,0,0,1,0-4,0.0,1.0
2,20090312431273200,2,831,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,0,0,1,0-4,1.0,1.0
3,20090312431273200,3,1147,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,0,0,1,0-4,0.0,1.0
4,20090312431273200,4,1863,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,0,0,1,0-4,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26296941,22100221145014656,1600,5483231,22,NaN,343.887291,36.701026,483.0,273.0,NaN,0,0,1,13-22,NaN,NaN
26296942,22100221145014656,1601,5485166,22,NaN,332.696070,141.493178,545.0,221.0,NaN,0,0,1,13-22,NaN,NaN
26296943,22100221145014656,1602,5485917,22,NaN,369.912859,140.569205,611.0,217.0,NaN,0,0,1,13-22,NaN,NaN
26296944,22100221145014656,1603,5486753,22,NaN,252.299653,123.805889,526.0,232.0,NaN,0,0,1,13-22,NaN,NaN


In [29]:
category_df

,session_id,index,event_name,name,fqid,room_fqid,text_fqid,level_group,correct,question
0,20090312431273200,0,cutscene_click,basic,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0-4,1.0,1.0
1,20090312431273200,1,person_click,basic,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,0.0,1.0
2,20090312431273200,2,person_click,basic,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,1.0,1.0
3,20090312431273200,3,person_click,basic,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,0.0,1.0
4,20090312431273200,4,person_click,basic,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
26296941,22100221145014656,1600,navigate_click,undefined,NaN,tunic.capitol_2.hall,NaN,13-22,NaN,NaN
26296942,22100221145014656,1601,navigate_click,undefined,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22,NaN,NaN
26296943,22100221145014656,1602,navigate_click,undefined,NaN,tunic.capitol_2.hall,NaN,13-22,NaN,NaN
26296944,22100221145014656,1603,navigate_click,undefined,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22,NaN,NaN


In [58]:
label_encoder = LabelEncoder()
label_encoder.fit(category_df['event_name'])
category_df['event_name'] = label_encoder.transform(category_df['event_name'])
label_encoder.fit(category_df['name'])
category_df['name'] = label_encoder.transform(category_df['name'])
label_encoder.fit(category_df['fqid'])
category_df['fqid'] = label_encoder.transform(category_df['fqid'])
label_encoder.fit(category_df['room_fqid'])
category_df['room_fqid'] = label_encoder.transform(category_df['room_fqid'])
label_encoder.fit(category_df['text_fqid'])
category_df['text_fqid'] = label_encoder.transform(category_df['text_fqid'])
label_encoder.fit(category_df['level_group'])
category_df['level_group'] = label_encoder.transform(category_df['level_group'])


LabelEncoder()

In [42]:
category_df['correct'] = category_df['correct'].ffill()

In [66]:
train_Classifier_x = category_df.drop('correct', axis=1)
train_Classifier_y = category_df.correct

In [67]:
X_Classifier_train, X_Classifier_val, y_Classifier_train, y_Classifier_val = train_test_split(train_Classifier_x, train_Classifier_y, test_size=0.3, random_state=42)
print(X_Classifier_train.shape, X_Classifier_val.shape, y_Classifier_train.shape, y_Classifier_val.shape)

(18407862, 9) (7889084, 9) (18407862,) (7889084,)


In [68]:
xgboost = XGBClassifier(max_depth=10,
                        n_estimators=50,
                        grow_policy='depthwise',
                        n_jobs=-1,
                        random_state=42,
                        tree_method='auto'
#                         enable_categorical=True
                       )

In [75]:
scaler = StandardScaler()

x_trainScaled = scaler.fit_transform(X_Classifier_train)
xgboost.fit(x_trainScaled, y_Classifier_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy='depthwise',
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=50, n_jobs=-1,
              num_parallel_tree=None, predictor=None, random_state=42, ...)

In [ ]:
print(X_Classifier_train.columns)
print(test_df.columns)
test_df = train_df.drop(columns=['elapsed_time', 'level','page','room_coor_x','room_coor_y','screen_coor_x','screen_coor_y','hover_duration','fullscreen','hq','music'])

In [82]:
label_encoder.fit(test_df['event_name'])
test_df['event_name'] = label_encoder.transform(test_df['event_name'])
label_encoder.fit(test_df['name'])
test_df['name'] = label_encoder.transform(test_df['name'])
label_encoder.fit(test_df['fqid'])
test_df['fqid'] = label_encoder.transform(test_df['fqid'])
label_encoder.fit(test_df['room_fqid'])
test_df['room_fqid'] = label_encoder.transform(test_df['room_fqid'])
label_encoder.fit(test_df['text_fqid'])
test_df['text_fqid'] = label_encoder.transform(test_df['text_fqid'])
label_encoder.fit(test_df['level_group'])
test_df['level_group'] = label_encoder.transform(test_df['level_group'])

In [99]:
# X_Classifier_test = pd.get_dummies(data=test_df)
X_Classifier_test = test_df[X_Classifier_train.columns[:]]
XGB_pred = xgboost.predict(X_Classifier_test)
XGB_pred = pd.DataFrame(XGB_pred)

New_df = pd.concat([train_df['session_id'], XGB_pred], axis=1)

In [100]:
New_df = New_df.rename(columns={0:'correct'})
New_df

,session_id,correct
0,20090312431273200,1
1,20090312431273200,0
2,20090312431273200,0
3,20090312431273200,0
4,20090312431273200,0
...,...,...
26296941,22100221145014656,1
26296942,22100221145014656,1
26296943,22100221145014656,1
26296944,22100221145014656,1


In [105]:
XGB_pred_dict = New_df.set_index(New_df.columns[0]).to_dict()[New_df.columns[1]]

In [106]:
XGB_submission = pd.read_csv('./sample_submission.csv')
XGB_submission["session"] = XGB_submission.session_id.apply(lambda x: int(x.split('_')[0]))
XGB_submission["question"] = XGB_submission.session_id.apply(lambda x: int(x.split('_')[-1][1:]))
XGB_submission

,session_id,correct,session_level,session,question
0,20090109393214576_q1,0,0,20090109393214576,1
1,20090312143683264_q1,0,3,20090312143683264,1
2,20090312331414616_q1,0,6,20090312331414616,1
3,20090109393214576_q2,0,0,20090109393214576,2
4,20090312143683264_q2,0,3,20090312143683264,2
5,20090312331414616_q2,0,6,20090312331414616,2
6,20090109393214576_q3,0,0,20090109393214576,3
7,20090312143683264_q3,0,3,20090312143683264,3
8,20090312331414616_q3,0,6,20090312331414616,3
9,20090109393214576_q4,0,1,20090109393214576,4


In [85]:
XGB_submission = XGB_submission.merge(New_df[['session_id','']])
XGB_submission['correct'] = XGB_pred

XGB_submission.to_csv(dataset + "XGB_first_submission.csv", index=False)

ValueError: Length of values (26296946) does not match length of index (54)